In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## IO markup

### Train

In [2]:
from modules.data import bert_data

In [3]:
train_df_path = "/home/eartemov/ae/work/factRuEval-2016/dev.csv"
valid_df_path = "/home/eartemov/ae/work/factRuEval-2016/test.csv"

In [4]:
device = "cuda:1"

In [5]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels2.txt",
    clear_cache=True,
    batch_size=8,
    device=device
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [6]:
from modules.models.bert_models import BERTBiLSTMAttnNCRF

In [8]:
model = BERTBiLSTMAttnNCRF.create(
    len(data.train_ds.idx2label), crf_dropout=0.3, nbest=len(data.train_ds.label2idx), is_freeze=False, hidden_dim=256,
    device=device)

build CRF...


In [9]:
from modules.train.train import NerLearner

In [10]:
num_epochs = 100

In [14]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTBiLSTMAttnNCRF-fit_BERT-IO.cpt",
    t_total=num_epochs * len(data.train_dl), lr=0.00001)

In [12]:
model.get_n_trainable_params()

179004667

In [ ]:
learner.fit(epochs=num_epochs)

### Predict

In [27]:
from modules.data.bert_data import get_data_loader_for_predict

In [28]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [29]:
preds = learner.predict(dl)

In [30]:
from sklearn_crfsuite.metrics import flat_classification_report

In [31]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [32]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [33]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=["I_ORG", "I_PER", "I_LOC"], digits=4)

In [34]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.8761    0.7224    0.7918      3865
       I_PER     0.9207    0.9342    0.9274      2112
       I_LOC     0.8767    0.8356    0.8556      1557

   micro avg     0.8902    0.8051    0.8456      7534
   macro avg     0.8911    0.8307    0.8583      7534
weighted avg     0.8887    0.8051    0.8430      7534



In [35]:
from modules.analyze_utils.main_metrics import precision_recall_f1

In [36]:
results = precision_recall_f1(true_labels, pred_labels)

processed 56409 tokens with 7534 phrases; found: 6814 phrases; correct: 6066.

precision:  89.02%; recall:  80.51%; FB1:  84.56

	LOC: precision:  87.67%; recall:  83.56%; F1:  85.56 1484

	ORG: precision:  87.61%; recall:  72.24%; F1:  79.18 3187

	PER: precision:  92.07%; recall:  93.42%; F1:  92.74 2143


